# Układy równań liniowych

### Pojęcia warte poznania

* Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
* Rząd macierzy: https://pl.wikipedia.org/wiki/Rząd_macierzy
* Kombinacja liniowa: https://pl.wikipedia.org/wiki/Kombinacja_liniowa
* Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney_*_ str. 245, pełny pseudokod: str. 252
* Pivoting: https://en.wikipedia.org/wiki/Pivot_element#Partial_and_complete_pivoting, K.C. str. 261, pełny pseudokod: str. 267
* Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
* Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
* Macierz dodatnio określona: https://pl.wikipedia.org/wiki/Określoność_formy
* Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
* Faktoryzacja Cholesky'ego: https://pl.wikipedia.org/wiki/Rozkład_Choleskiego, K.C. str. 305

Dodatkowo:
* Wskaźnik uwarunkowania: https://pl.wikipedia.org/wiki/Wska%C5%BAnik_uwarunkowania, K.C. str.321
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method, K.C. 323

Książka dla wytrwałych (naprawdę): Y. Saad "Iterative Methods for Sparse Linear Systems"

In [2]:
from typing import Optional, Tuple
import numpy as np
import pixiedust
import itertools
from copy import deepcopy

Pixiedust database opened successfully


### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

**Pytanko:** jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

**Zadanko:** Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do ["alfa-konwersji"](https://pl.wikipedia.org/wiki/Konwersja_α))

In [3]:
def agh_superfast_matrix_multiply(a: np.array, b: np.array) -> np.array:
    """Perform totally ordinary multiplication of matrices.
    Using np.arrays since matrices are deprecated
    :param a: matrix with dimensions M by N
    :param b: matrix with dimensions N by O
    :return:  matrix with dimensions M by O
    """
    M, N = a.shape
    X, O = b.shape
    if N != X: raise ValueError("Matrices must be properly shaped")
    result = np.zeros((M, O))
    for i in range(M):
        for j in range(O):
            acc: np.float128 = .0
            for k in range(N):
                acc += a[i][k] * b[k][j]
            result[i][j] = acc
    return result

m1 = np.array([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.array([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, np.matmul(m1, m2)), "Wrong multiplication result"


### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki:

In [4]:
def gaussian_elimination(A: np.array, b: np.array) -> np.array:
    if A.shape[0] != A.shape[1]: raise ValueError("Matrix A must be square")
    n:int = len(A)
    for i in range(n - 1):
        for j in range(i + 1, n):
            row_factor = A[j][i]/A[i][i]
            A[j][i] = .0
            for k in range(i + 1, n):
                A[j][k] -= row_factor * A[i][k]
            b[j] -= row_factor * b[i]

    for i in range(n - 1, -1, -1):
        for j in range(i + 1, n):
            b[i] -= A[i][j] * b[j]
        b[i] *= 1.0/A[i][i]
    return b

In [5]:
def gauss_with_spp(A: np.array, b: np.array) -> np.array:
    """
    Gaussian elimination with scaled partial pivoting
    """
    if A.shape[0] != A.shape[1]: raise ValueError("Matrix A must be square")
    n:int = len(A)
    scale: np.array = np.zeros(n)
    for i,j in itertools.product(range(n), range(n)):
        scale[i] = max(scale[i], np.abs(A[i][j]))
    ind: np.array = np.arange(n)
    for i in range(n - 1):
        max_ratio: np.float32 = .0
        pivot = i
        for j in range(i,n):
            if np.abs(A[ind[j], i]) / scale[ind[j]] > max_ratio:
                max_ratio = np.abs(A[ind[j], i]) / scale[ind[j]]
                pivot = j
        ind[i], ind[pivot] = ind[pivot], ind[i]
        for j in range(i + 1, n):
            row_factor = - A[ind[j], i] / A[ind[i], i]
            A[ind[j], i] = .0
            for k in range(i+1, n):
                A[ind[j], k] += row_factor * A[ind[i], k]
            b[ind[j]] += row_factor * b[ind[i]]

    for i in range(n-1, -1, -1):
        for j in range(i + 1, n):
            b[ind[i]] -= A[ind[i], j] * b[ind[j]]
        b[ind[i]] /= A[ind[i], i]
    result = np.zeros(n)
    for i in range(n):
        result[i] = b[ind[i]]
    return result

In [6]:
# %%pixie_debugger
A = np.array([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.transpose([9.5740, 7.2190, 5.7300, 6.2910])

x0 = np.linalg.solve(deepcopy(A), np.copy(b))
x1 = gaussian_elimination(deepcopy(A), np.copy(b))
x = gauss_with_spp(deepcopy(A), np.copy(b))

**Pytanie**: dlaczego wołamy `transpose()` na wektorze `b`?

Sprawdźmy, czy rozwiązanie jest ok (**Pytanie'**: dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?):

In [7]:
print(np.allclose(np.dot(A, x), b))
print(np.allclose(np.dot(A, x1), b))
print(np.allclose(x1, x0))

True
True
True


### Zadania, c.d.

5. Zaimplementować algorytm faktoryzacji LU macierzy
6. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
7. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [27]:
def agh_superfast_lu(A: np.array) -> Optional[Tuple[np.array, np.array]]:
    """Perform LU decomposition of a matrix.
    
    :param a: LU-decomposable matrix (without row swapping)
    :return:  Tuple of L and U matrices
    """
    if A.shape[0] != A.shape[1]: raise ValueError("Matrix A must be square")
    n = len(A)
    L = np.zeros((n,n))
    U = deepcopy(A)
    for i in range(n - 1):
        L[i][i] = 1.0
        for j in range(i + 1, n):
            if U[i][i] == 0: raise ValueError("Matrix is not LU - decomposable")
            row_factor = U[j][i]/U[i][i]
            U[j][i] = .0
            L[j][i] = row_factor
            for k in range(i + 1, n):
                U[j][k] -= row_factor * U[i][k]
    L[n-1][n-1] = 1.0
    return L,U
    
def agh_superfast_check_spd(A: np.array) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    """
    return np.allclose(np.transpose(A), A) and np.all(np.linalg.eigvals(A) > 0)
    

def agh_superfast_cholesky(A: np.array) -> Optional[np.array]:
    """Perform a Cholesky decomposition of a matrix.
    
    :param A: Symetric and positively defined matrix
    :return:  L matrix such that L*L^-1 == A
    """
    if not agh_superfast_check_spd(A): raise ValueError("Matrix A must be symetric and positively defined")
    n = len(A)
    L = np.zeros((n,n))
    for i in range(n):
        for j in range(i):
            L[i][j] = A[i][j]
            for k in range(j):
                L[i][j] -= L[i][k] * L[j][k]
            L[i][j] /= L[j][j]
        L[i][i] = A[i][i]
        for k in range(i):
            L[i][i] -= L[i][k]*L[i][k]
        L[i][i] = np.sqrt(L[i][i])
    return L

In [29]:
A = np.array([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

B = np.array([[ 6,  3,  4,  8],
               [ 3,  6,  5,  1],
               [ 4,  5, 10,  7],
               [ 8,  1,  7, 25]])

L, U = agh_superfast_lu(A)
print(np.allclose(A, np.dot(L,U)))
print(agh_superfast_check_spd(B))
C = agh_superfast_cholesky(B)
print(np.allclose(B, np.dot(C, np.transpose(C))))

True
True
True


### Zadania, opcjonalnie
5. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
6. za pomocą tejże metody rozwiązać powyższy układ równań

// będą laby o tym, więc teraz nie robię

\*  wszystkie referencje odnoszą się do [książki](https://wiki.iiet.pl/lib/exe/fetch.php?media=studia:przedmioty:mownit:numerical_mathematics_and_computing.pdf) David Kincaid, Ward Cheney - "Numerical Mathematics and Computing, 6th edition"
\** _ulubiony_ język programowania staramy się pojmować rozsądnie, tj. z wyłączeniem języków pokroju Prologa, języków z [tej listy](https://en.wikipedia.org/wiki/Esoteric_programming_language) oraz Assemblera i PHP. Haskella można używać na własną odpowiedzialność.